# Demo 1: Fallzahlen, Genesungen, und Todesfällen Vergleichen

## Importing Dependencies

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wgts
from IPython.display import display

## Loading Datasets

In [4]:
# test data read
covid_19_dataset = pd.read_csv('../dataset/time-series-19-covid-combined.csv')
regions_dataset = pd.read_csv('../dataset/Countries by continents.csv')

## Creating Widgets

### Creating Countries & Continents Dropdown Menus

In [23]:
countries_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Country'].unique(),
    description='Country:',
    disabled=False,
)

continent_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Continent'].unique(),
    description='Continent:',
    disabled=False,
)

### Creating Regional Category Radio Button

In [24]:
regional_category_radio = wgts.RadioButtons(
    options=['Country', 'Region', 'World'],
    description='Regional Category:',
    disabled=False
)

### Creating Time Category Radio Button

In [ ]:
time_category_radio = wgts.RadioButtons(
    options=['Daily', 'Weekly', 'Monthly', 'Yearly', 'Complete'],
    description='Time Category:',
    disabled=False
)

### Creating Months, Years Dropdown Menus & Day DatePicker

In [37]:
import datetime
# minimum date value in dataset
dateMin = datetime.datetime.strptime(covid_19_dataset['Date'].min(), '%Y-%m-%d').date()
# maximum date value in dataset
dateMax = datetime.datetime.strptime(covid_19_dataset['Date'].max(), '%Y-%m-%d').date()

day_datepicker = wgts.DatePicker(
    description='Day:',
    disabled=False,
    max=dateMax,
    min=dateMin
)

year_dropdown_menu = wgts.Dropdown(
    options= range(dateMin.year, dateMax.year + 1),
    description='Year:',
    disabled=False,
)

month_dropdown_menu = wgts.Dropdown(
    options= ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"],
    description='Month:',
    disabled=False,
)

### Creating Reload Button

In [43]:
reload_button = wgts.Button(
    description='Reload',
    disabled=False,
    button_style='info',
    tooltip='Reload',
    icon='check'
)